# Single-peak mesh fitting

Runs a parallelized peak fitting routine on xrd maps

In [ ]:
#Import necessary libaries
%matplotlib widget
import os
import sys
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
#To import DanMAX from the folder above:
sys.path.append('../')
import DanMAX as DM
    
style = DM.darkMode(style_dic={'size':'large'})

def apply_roi(maps, bkg_roi, peak_roi,local_bkg=True):
    """apply a region of interest to diffraction data. Subtract local
    background of specified."""
    if local_bkg:
        xrd_map = (maps['xrd_map'][:,:,peak_roi].transpose(2,0,1) -np.mean(maps['xrd_map'][:,:,bkg_roi],axis=2)).transpose(1,2,0)
    else:
        xrd_map = maps['xrd_map'][:,:,peak_roi]
    tth = maps['x_xrd'][peak_roi]
    return xrd_map, tth

### Load the data
Set the group and sample from the sample database (see *Sample_list.ipynb*)  
or use a list of scan numbers

In [ ]:
# Define scan location:
#Note that scans must be a list!

samples_database = False
if samples_database: # To use samples_database, and to do bulk processing, use the "Samples_list" notebook
    groups = DM.getProposalScans()
    group = 'group'
    sample = 'sample'
    scans = groups[group][sample]
    title = f'{group}: {sample}'
else:
    scans = [200]
    title = DM.getScan_id(DM.findScan(scans[0]))
proposal,visit=DM.getCurrentProposal()
 
#Select ranges to load (in the unit the data were integrated with)
#useful for reducing the size of large datasets
xrd_range = None # (lower,upper) or None
azi_range = None # (lower,upper) or None

#Load data
maps = DM.mapping.stitchScans(scans,XRF=False,proposal=proposal,visit=visit,xrd_range=xrd_range,azi_range=azi_range)
#Apply I0 correction
maps['xrd_map'] = (maps['xrd_map'].transpose(2,0,1)/ maps['I0_map']).transpose(1,2,0)

### Define a mask and ROI
Use the interactive mask to select a region of interest with left/right mouse click in the upper plot.  
Select a mask threshold with left/right mouse click in the histogram to the right.

In [ ]:
imask = DM.InteractiveMask(maps['xrd_map'],reduction_mode='std')

In [ ]:
# get the current mask(s) from the interactive tool
mask = imask.getMask()
bool_mask = mask.astype(bool)
nan_mask = imask.getNanMask()
# get the lower/upper threshold
lower, upper = sorted(imask.getRoiThreshold())

########################
apply_local_bkg = False
num_of_bgr_pts = 5
########################
# set the number of points to use as background
peak_roi = np.s_[lower+num_of_bgr_pts:upper-num_of_bgr_pts]
bkg_roi = np.r_[lower:lower+num_of_bgr_pts,upper-num_of_bgr_pts:upper] # indexes from background
xrd_map,tth = apply_roi(maps,bkg_roi,peak_roi,local_bkg=apply_local_bkg)

# calculate the mean diffraction signal for the selected mask and ROI
y_mean = np.nanmean(xrd_map[bool_mask],axis=0)

# plot the mask and selected ROI
fig, [ax0,ax1] = plt.subplots(1,2)
fig.suptitle(title)
ax0.set_title('mask')
ax0.imshow(mask)
ax0.set_xticks([])
ax0.set_yticks([])

ax1.set_title('roi')
ax1.plot(tth,y_mean)
ax1.plot(tth,y_mean)

ax1.axvspan(tth[0],tth[num_of_bgr_pts],color='#AAAAAA44')
ax1.axvspan(tth[-num_of_bgr_pts],tth[-1],color='#AAAAAA44')

## Run mesh peak fit.
Run the parallelized peak fitting and show a short summary of the results

In [ ]:
fitmap = DM.fitting.fitMesh(DM.fitting.pVPeakFit,  # peak fit function
                            tth,                   # x-values
                            xrd_map,                  # list of list of y-values
                            mask,                  # mask of px to fit (int mask)
                            verbose=True,          # print output
                            sequential=True,       # toggle sequential mode
                            fun_kwargs={}          # peak function keyword arguments
                           )

print('\n',f'Fitting complete - {np.sum(mask):d} pixels fitted\n')
# parse the fitting results to a more meaningfull
# dictionary of results, based on the scheme of
# the parsing function
parsed_params,names = DM.fitting.parseFitMesh(DM.fitting.parsePVPeakFit, # parsing function
                                           fitmap,                    # results from the mesh fit
                                          )
results = {name:parsed_params[:,:,i] for i,name in enumerate(names)}

# print a short summary of the results
print(f"{' parameter':<15}{'mean':>15}{'std':>15}\n",''.join(['‾']*45))
for key,val in results.items():
    print(f' {key:<15} {np.nanmean(val[bool_mask]):14.4f} {np.nanstd(val[bool_mask]):14.4f}')

### plot the results as maps

In [ ]:
# Set the number of columns for the figure
cols = 4

# initialize figure
rows = int(len(results)/cols) + (len(results)%cols!=0)
fig, axes = plt.subplots(rows,cols,sharex=True,sharey=True)
fig.suptitle('Results')
axes = axes.flatten()
i=0
for name,result in results.items():
    ax = axes[i]
    ax.set_title(name)
    ax.imshow(result*nan_mask)
    i +=1
    
ax.set_xticks([])
ax.set_yticks([])

# delete surplus plots
for i in range(1,cols*rows-len(results)+1):
    fig.delaxes(axes[-i])
fig.tight_layout()

In [ ]:
# calculated peak values (without background)
y_calc = DM.fitting.pVPeakMesh(tth,
                                results['position'],
                                results['integral'],
                                results['FWHM'],
                                results['eta'],
                               )
# calculated background
bgr_coeff = np.array([results[key] for key in sorted(results.keys()) if 'bgr_' in key])
bgr =  np.polynomial.chebyshev.chebval(tth,bgr_coeff)


## Save the results
Change the `save_results` flag to save

In [ ]:
#####################
save_results = False
#####################

if save_results:
    if samples_database:
        save_dst = DM.findScan(scans[0]).split('raw/')[0]+f'process/mesh_fit/{group}/{sample}'
    else:
        save_dst = DM.findScan(scans[0]).replace('/raw','/process/mesh_fit')
    
    if not os.path.isdir(os.path.dirname(save_dst)):
        os.makedirs(os.path.dirname(save_dst))
    
    h5.get_config().track_order=True
    with h5.File(save_dst,'w') as f:
        f.create_dataset('x_map',data=maps['x_map'])
        f.create_dataset('y_map',data=maps['y_map'])
        f.create_dataset('mask',data=mask)
        f.create_dataset('obs',data=xrd_map)
        f.create_dataset('calc',data=y_calc+bgr)
        f.create_dataset('diff',data=xrd_map-(y_calc+bgr))
        f.create_dataset('bgr',data=bgr)
        for name, result in results.items():
            f.create_dataset(name,data=result)
